<h1>SASSIE Waveglider Data Use and Visualization</h1>
<p> The Wavegliders are automatic/remotely piloted instruments that take measurements of physical ocean and atmospheric data in the open water. Waveglider data are downloaded and plotted using the code below! <b>Please run the 'Data Download and Metadata Viewing' and 'Supporting Code' sections in order before running the 'Figure Making Code'.</b> </p>
    
<p>The aim of this notebook is to assist the end user in exploratory data analysis by downloading the SASSIE data from NASA's PODAAC, opening the dataset and displaying it's associated metadata, and creating a few visualizations. This notebook was created by Elizabeth Westbrook. For questions and trouble shooting, please email westbrooke@uncw.edu.</p>

# Data Download and Metadata Viewing

The code in this section will download the dataset from PO.DAAC and open it as an xarray object for metadata and variable attribute viewing.

In [ ]:
import numpy as np
import xarray as xr
import glob 
from datetime import datetime, timedelta 
import matplotlib.pyplot as plt
import cartopy
import matplotlib
import os
import sys
import pandas as pd
from mpl_toolkits import mplot3d
import math
import requests

<h2>Download The Waveglider data from PO.DAAC</h2>
<p> All data from the SASSIE campaign is stored on NASA's PO.DAAC. The code in this section of the notebook will download SASSIE Waveglider data from PO.DAAC, which is accessed through earthdata. If you do not already have an earthdata account, you can create one <a href="https://urs.earthdata.nasa.gov/">here</a> . </p>
    <p>Please enter your earthdata credentials  below.</p>

In [ ]:
username = 'your_username'
password = 'your_password'
earthdata = requests.auth.HTTPBasicAuth(username, password)

## DIRECTORY TO DATA
dir_in = 'Data/wavegliders/' 

The next block of code creates the directory specified above and downloads the waveglider files to your binder session if it has not already been downloaded.<b> To download the dataset to your local disk from here, right click on the file you want to download and click 'download'. </b>

In [ ]:
##SASSIE DATA DIR
if not os.path.exists(dir_in):
    os.makedirs(dir_in)

wg_numbers = ['130','153','247','245']
for number in wg_numbers:
    filename = 'SASSIE_Fall_2022_Waveglider_'+number+'.nc'
    if os.path.isfile(dir_in+filename)==False:
        url = 'https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/SASSIE_L2_WAVEGLIDERS_V1/'+filename
        with requests.Session() as session:
                session.auth = (username, password)
                r1 = session.request('get', url)
                r = session.get(r1.url, auth=(username, password))
                if r.status_code == 200:
                    if r.ok:
                        with open(dir_in+filename, 'wb') as f:
                            f.write(r.content) 
                            print('Saving Input File: ' + dir_in+filename) 
                else:
                    print("Error:", r.status_code)
                    if r.status_code == 401:
                        print ('Your Username and/or password are incorrect. Please try again')
    else: 
        print('Waveglider '+ number+' file is already in local directory')

<h2> View The Metadata Inside the Waveglider Files</h2>

The netCDF file has global metadata attributes and attributes associated with each variable. This next block will load data and metadata of the netCDF file into an xarray object (ds). <br> The data set will then be displayed in a clickable HTML format. 

In [ ]:
#See information about the entire dataset:
print('Displaying just 1/4 waveglider files')
files = sorted(glob.glob(dir_in + '/*.nc'))
ds = xr.open_dataset(files[0])
ds

# Supporting Code 
The code in this section provides a set up for the figure making code below by defining directories for data and figures and creating functions that will be called to actually map the data

<h3>Create a Directory to Save Figures</h3>

The next block creates a directory for figures to be saved

In [ ]:
#LOCAL DIRECTORY TO SAVE FIGURES
fig_dir ='Figures/waveglider/'
#FIGURE DIR 
if not os.path.exists(fig_dir):
    os.makedirs(fig_dir)

<h3>Define a Colormap and Label for Each Variable in the File</h3>

Within SASSIE's collection of jupyter notebooks, the colormaps used for each variable are held as consistant as possible across all datasets. This function defines the colormap and a label for the variable of interest. 

In [ ]:
#DEFINES COLORMAPS AND LABELS OF EACH VARIABLE IN THIS DATA SET
def define_variable_attributes(var):
    if var == 'surface_wave_period':
        colormap = 'cool'  
        var_label = 'Surface Wave Period (s)'
    if (var=='wind_speed'): 
        colormap = 'cividis'
        var_label = 'Wind Speed (m/s)'
    if (var=='wind_direction'):
        colormap = 'twilight'
        var_label = 'Wind Direction ($^{\circ}$N)'
    if (var =='air_temperature'):
        colormap = 'magma'
        var_label = 'Air Temperature ($^{\circ}$C)'
    if (var == 'air_pressure'):
        colormap = 'Blues'
        var_label= 'Air Pressure (hPa)'
    if var == 'surface_wave_direction':
        cmap = 'hsv'
        var_label = 'Surface Wave Direction ($^{\circ}$N)'
    if var=='surface_wave_height':
        colormap = 'autumn'
        var_label = 'Wave Height (m)'
    if var =='salinity':
        colormap = 'viridis'
        var_label = 'Salinity'
    if var =='water_temperature':
        colormap = 'plasma'
        var_label = 'Water Temperature ($^{\circ}$C)'
    if var == 'time':
        colormap = 'jet'
        var_label = 'Date'
    return colormap,var_label

<h3>Define a Function to Create a Map of the Study Area</h3>

The following function creates a map of the SASSIE study area, which is defined by minimum and maximum lat/lon values. These ranges can be changed later when the function is called to zoom in/out on the study area.

In [ ]:
def map_study_area(latmin,latmax,lonmin,lonmax):
    
    global fig 
    global ax
    
    #create the map as a figure, set the lat and lon ranges, and add land + river data:
    fig = plt.figure(figsize=(10,8))
    ax = plt.axes(projection=cartopy.crs.NorthPolarStereo(central_longitude=-150))
    ax.set_extent([lonmin,lonmax,latmin,latmax], crs=cartopy.crs.PlateCarree())
    ax.coastlines(color='k')  
    ax.add_feature(cartopy.feature.LAND, facecolor = '0.50',zorder=1)
    ax.add_feature(cartopy.feature.RIVERS,facecolor='blue')
    #Add lat and lon gridlines and labels:
    gl = ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False, alpha=0.3) #draw_labels=True gives lat labels.
    gl.ylocator = matplotlib.ticker.FixedLocator(np.arange(60,90,1))
    gl.xlocator = matplotlib.ticker.FixedLocator(np.arange(-180,180,2))
    gl.top_labels = False
    gl.bottom_labels = True 
    gl.right_labels = False
    
    #Add markers for reference cities on the coast:
    if (latmin<71.2906) & (lonmin<-156.7886):
        utqiagvik = ax.scatter(-156.7886,71.2906,s=100,transform=cartopy.crs.PlateCarree(),c='red',marker = '*',label='Utqiagvik, AK',zorder=2)
    if (latmin<70.2002) & (lonmax>-148.4597):
        deadhorse = ax.scatter(-148.4597,70.2002,s=100,c='cyan',transform=cartopy.crs.PlateCarree(),marker = '*',label='Deadhorse, AK',zorder=2)
        

<h3>Configure Supporting Data to Add to Maps</h3>

<p>The functions for viewing and plotting this data set below have options to include bathymetry, ice,  and/or shiptrack data to add context to maps. If you are using these options, run
    <br>the following code blocks to:
    <br>1. Create a directory for SASSIE Ship Track data and acess bathymetry data from NOAA
    <br>2. Define functions that add these data to your map when called.</p>

<h4>Create Directory for Shiptrack Data and Access Bathymetry Data </h4>

In [ ]:
## DIRECTORY TO SHIP TRACK DATA
ship_dir =  'Data/TSG/' 

#DOWLOAD SHIPTRACK DATA
if not os.path.exists(ship_dir):
    os.makedirs(ship_dir)
    url = 'https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/SASSIE_L2_SHIPBOARD_TSG_V1/SASSIE_Fall_2022_Shipboard_TSG.nc'
    with requests.Session() as session:
            session.auth = (username, password)
            r1 = session.request('get', url)
            r = session.get(r1.url, auth=(username, password))
            if r.status_code == 200:
                if r.ok:
                    with open(ship_dir+'SASSIE_Fall_2022_Shipboard_TSG.nc', 'wb') as f:
                        f.write(r.content) 
                        print('Saving Input File: ' + ship_dir+'SASSIE_Fall_2022_Shipboard_TSG.nc') 
            else:
                print("Error:", r.status_code)
                if r.status_code == 401:
                    print ('Your Username and/or password are incorrect. Please try again')
else: 
    print('Shipboard TSG file is already in local directory')
    
    
#READ IN TOPOGRAPHY/BATHYMETRY DATA
url = 'http://ferret.pmel.noaa.gov/thredds/dodsC/data/PMEL/etopo2.nc'
etopodata = xr.open_dataset(url) 

<h4>Define a Function to Index Relevant Bathymetry Data and Add it to the Map</h4>
This function will index bathymetry data from NOAA within the appropriate spatial range and add it to the map. 


In [ ]:
def add_bathy_data(latmin,latmax,lonmin,lonmax):
        topoin = etopodata.rose.values[0:-1:5,1:-1:5]
        lons = etopodata.etopo2_x.values[0:-1:5]
        lats = etopodata.etopo2_y.values[0:-1:5]
        lons_in_range = lons[np.where((lons >lonmin-1) & (lons<lonmax+1))]
        lats_in_range = lats[np.where((lats >latmin-1) & (lats<latmax+1))]
        topo_in_range = np.squeeze(topoin[np.squeeze(np.where((lats >latmin-1) & (lats<latmax+1))),:][:,np.where((lons >lonmin-1) & (lons<lonmax+1))])
        [bathy_lon,bathy_lat] = np.meshgrid(lons_in_range,lats_in_range)
        
        bathy = ax.contour(bathy_lon,bathy_lat,topo_in_range,np.arange(-6000,-1000,300),transform=cartopy.crs.PlateCarree(),cmap='gray',alpha = 0.2,zorder = 0)

<h4>Define a Function to Add Shiptrack Data to a Map</h4>
This function will pull the lat/lon data from the SASSIE Shipboard TSG file and put it onto a map. 

In [ ]:
def add_ship_track():
        ds_ship = xr.open_dataset(ship_dir+'/SASSIE_Fall_2022_Shipboard_TSG.nc')
        ship_time = np.squeeze(ds_ship['time'])
        ship_lat = np.squeeze(ds_ship['latitude'])
        ship_lon = np.squeeze(ds_ship['longitude'])
       
        track = ax.plot(ship_lon, 
                     ship_lat,linewidth = 0.5,
                     c='black',
                       transform=cartopy.crs.PlateCarree(),label = 'Ship Track',zorder=1)

# Figure Making Code

<h2>Make Maps of This Data</h2>
<p>Using the code in this section, the user can plot the Waveglider data on a map colored by time, waveglider number, or one of the pyhsical measurement variables recorded. </p>

<h3> Mapping the Time and Location of Waveglider Data Collection</h3>

The map_wavegliders function shows the track of the Wavegliders on a map, colored by time. Various features of this function:
    <br>1. The function automatically adds the track of the R/V Woldstad in Black for reference. 
    <br>2. This function adds bathymetry contours showing the position of the continental shelf by default for reference. 

In [ ]:
##USE THIS TO PLOT THE WAVE GLIDER TRACKS ON THE WHOLE CAMPAIGN MAP AND COLOR BY TIME##
def map_wavegliders(bathymetry_data=True,ship_track=True):
        
    ##ESTABLISH START AND END TIMES FOR WG COLOR SCALE
    ds = xr.open_dataset(files[3])
    time = ds.time.values
    wg_start_time = min(time)
    wg_end_time = max(time)
    
    ##CREATE A MAP WITH LAND AND CITY MARKERS
    latmin = 70
    latmax =74
    lonmin=-157
    lonmax=-142
    
    #make the map
    map_study_area(latmin,latmax,lonmin,lonmax)
    
    #give it a title
    ax.set_title('Waveglider Tracks',fontsize=22,pad=1) 

    
    ##OPTIONAL MAP ADD-ONS
    if ship_track==True:
        add_ship_track()
    
    if bathymetry_data==True:
        add_bathy_data(latmin,latmax,lonmin,lonmax)
        
    ##APPLY THE TIME AND LOCATION DATA IN EACH WAVEGLIDER FILE 
    for file in files:
        ds_wg = xr.open_dataset(file)
        wg_time = np.squeeze(ds_wg['time'])
        wg_lat = np.squeeze(ds_wg['latitude'])
        wg_lon = np.squeeze(ds_wg['longitude']) 
        
        deployment_track = ax.scatter(wg_lon,wg_lat,s = 1,
                       c = wg_time,cmap = 'jet',
                       transform=cartopy.crs.PlateCarree(),zorder=2,vmin = wg_start_time.astype('int64'),vmax = wg_end_time.astype('int64'))
                        
    cbar = fig.colorbar(deployment_track, ax=ax, orientation="horizontal", pad=0.1)
    cbar.set_label(label='Date',size='large',weight='bold')
    cbar_tick_array=(np.linspace(wg_start_time.astype('int64'),wg_end_time.astype('int64'),5))
    cbar.set_ticks(cbar_tick_array)
    cbar.set_ticklabels(pd.to_datetime(cbar_tick_array).date)
        
    ##SAVE THE FIGURE 
    if not os.path.exists(fig_dir+'map'):
        os.makedirs(fig_dir+'map')
    print('Saving Output Image:  '+fig_dir+'map/waveglider_tracks_map.png')
    plt.savefig(fig_dir+'map/waveglider_tracks_map.png',dpi='figure',format='png')

In [ ]:
map_wavegliders()

<h3> Mapping the Tracks of Individual Wavegliders</h3>

The individual_tracks function shows the track of the Wavegliders on a map, colored by waveglider number. Various features of this function:
    <br>1. The function automatically adds the track of the R/V Woldstad in Black for reference. 
    <br>2. This function adds bathymetry contours showing the position of the continental shelf by default for reference. 

In [ ]:
##USE THIS TO PLOT THE WAVE GLIDER TRACKS ON THE WHOLE CAMPAIGN MAP AND COLOR BY WG NUMBER

def map_individual_tracks(wg_number='all',bathymetry_data=True,ship_track=True):
    
    ##CREATE A MAP WITH LAND AND CITY MARKERS
    latmin = 70
    latmax =74
    lonmin=-157
    lonmax=-142
    
    map_study_area(latmin,latmax,lonmin,lonmax)

    ax.set_title('Waveglider Tracks',fontsize=22,pad=1) 
    
    ##OPTIONAL MAP ADD-ONS
    if ship_track==True:
        add_ship_track()
        
    if bathymetry_data==True:
        add_bathy_data(latmin,latmax,lonmin,lonmax)
        
    ##ADD LOCATION DATA OF THE SPECIFIED WAVEGLIDERS
    if (wg_number =='all')|(wg_number==130):
        ds_wg = xr.open_dataset(files[0])
        track = ax.plot(ds_wg.longitude, 
                ds_wg.latitude,linewidth = 1,
                c='blue',
                transform=cartopy.crs.PlateCarree(),label = 'Waveglider 130',zorder=1)
    if (wg_number =='all')|(wg_number==153):
        ds_wg = xr.open_dataset(files[1])
        track = ax.plot(ds_wg.longitude, 
                ds_wg.latitude,linewidth = 1,
                c='red',
                transform=cartopy.crs.PlateCarree(),label = 'Waveglider 153',zorder=1)
    if (wg_number =='all')|(wg_number==245):
        ds_wg = xr.open_dataset(files[2])
        track = ax.plot(ds_wg.longitude, 
                ds_wg.latitude,linewidth = 1,
                c='green',
                transform=cartopy.crs.PlateCarree(),label = 'Waveglider 245',zorder=1)
    if (wg_number =='all')|(wg_number==247):
        ds_wg = xr.open_dataset(files[3])
        track = ax.plot(ds_wg.longitude, 
                ds_wg.latitude,linewidth = 1,
                c='cyan',
                transform=cartopy.crs.PlateCarree(),label = 'Waveglider 247',zorder=1)
    
    
    plt.legend(loc=2)

    ##SAVE THE FIGURE 
    if not os.path.exists(fig_dir+'map'):
        os.makedirs(fig_dir+'map')
    print('Saving Output Image:  '+fig_dir+'map/waveglider_number_tracks_map.png')
    plt.savefig(fig_dir+'map/waveglider_number_tracks_map.png',dpi='figure',format='png')

In [ ]:
map_individual_tracks()
map_individual_tracks(wg_number = 245)

<h3>Mapping Physical Measurement Sea Surface and Atmospheric Waveglider Data </h3>

<p>The map_waveglider_variable function shows the track of the Wavegliders on a map, colored by a physical measurement variable they recorded. Various features of this function: </p>
    <p>1. The user can specify a specific waveglider (130,153,245,247) of plot all WGs
    <br>2. This function adds bathymetry contours showing the position of the continental shelf by default for reference. 
    <br>4. The user must specify the variable to be plotted, as well as the minimum and maximum value for that variable.</p>

In [ ]:
##USE THIS TO MAP WAVEGLIDER VARIABLES 
def map_waveglider_variable(var,var_min,var_max,wg_number='all',ship_track=True,bathymetry_data=True):
    
    if wg_number=='all':
        ##LOAD IN ALL WAVEGLIDER DATA
        ds_130 = xr.open_dataset(dir_in+'SASSIE_Fall_2022_Waveglider_130.nc')
        ds_153 = xr.open_dataset(dir_in+'SASSIE_Fall_2022_Waveglider_153.nc')
        ds_245 = xr.open_dataset(dir_in+'SASSIE_Fall_2022_Waveglider_245.nc')
        ds_247 = xr.open_dataset(dir_in+'SASSIE_Fall_2022_Waveglider_247.nc')
        wg_time = ds_245['time']
    else:
        ##LOAD IN DATA FROM SELECTED WAVEGLIDER
        file = dir_in+'SASSIE_Fall_2022_Waveglider_'+str(wg_number)+'.nc'
        ds = xr.open_dataset(file)
        wg_time = ds['time']
    
    ##CREATE A MAP WITH LAND AND CITY MARKERS  
    latmin = 70
    latmax =74
    lonmin=-158
    lonmax=-142
    
    colormap,var_label = define_variable_attributes(var)
          
    map_study_area(latmin,latmax,lonmin,lonmax)
    
    if wg_number=='all':
        ax.set_title('All Waveglider '+var_label+' Readings',fontsize=15)
    else:
        ax.set_title('Waveglider '+str(wg_number)+' '+var_label+' Readings',fontsize=15)
   
    ##OPTIONAL MAP ADD-ONS
    if ship_track==True:
        add_ship_track()
        
    if bathymetry_data ==True:
        add_bathy_data(latmin,latmax,lonmin,lonmax)
        
    if wg_number=='all':
        if (var=='salinity')|(var=='water_temperature'):
            track_130 = ax.scatter(ds_130['longitude'], 
                     ds_130['latitude'],s=4, 
                     c=ds_130[var][0,:],cmap=colormap,
                       transform=cartopy.crs.PlateCarree(),
                          vmin = var_min,
                          vmax = var_max,zorder = 2)
            
            track_153 = ax.scatter(ds_153['longitude'], 
                     ds_153['latitude'],s=4, 
                     c=ds_153[var][1,:],cmap=colormap,
                       transform=cartopy.crs.PlateCarree(),
                          vmin = var_min,
                          vmax = var_max,zorder = 2)
            
            track_245 = ax.scatter(ds_245['longitude'], 
                     ds_245['latitude'],s=4, 
                     c=ds_245[var][0,:],cmap=colormap,
                       transform=cartopy.crs.PlateCarree(),
                          vmin = var_min,
                          vmax = var_max,zorder = 2)
            
            track_247 = ax.scatter(ds_247['longitude'], 
                     ds_247['latitude'],s=4, 
                     c=ds_247[var][0,:],cmap=colormap,
                       transform=cartopy.crs.PlateCarree(),
                          vmin = var_min,
                          vmax = var_max,zorder = 2)
            
            matplotlib.pyplot.colorbar(track_247, ax=ax, orientation="horizontal", pad=0.1).set_label(label=var_label,size='large',weight='bold')


    else:  
        if (var=='salinity')|(var=='water_temperature'):
            contour = ds[var][0,:]
        elif (var=='salinity') & (wg_number==153):
            contour = ds[var][1,:]
        else: 
            contour = ds[var]
        ##APPLY WAVEGLIDER DATA 
        wg_track = ax.scatter(ds['longitude'], 
                     ds['latitude'],s=5, 
                     c=contour,cmap=colormap,
                       transform=cartopy.crs.PlateCarree(),
                          vmin = var_min,
                          vmax = var_max,zorder = 2)

        plt.legend(loc=2)           
        matplotlib.pyplot.colorbar(wg_track, ax=ax, orientation="horizontal", pad=0.1).set_label(label=var_label,size='large',weight='bold')
    
    
    #SAVE FIGURE 
    if not os.path.exists(fig_dir+ var):
        os.makedirs(fig_dir+var)
    print('Saving Output Image:  '+fig_dir+var+'/Waveglider_'+str(wg_number)+'.png')
    plt.savefig(fig_dir+var+'/Waveglider_'+str(wg_number)+var+'.png',dpi='figure',format='png')
    
    
    

In [ ]:
map_waveglider_variable('salinity',23,30)
map_waveglider_variable('water_temperature',-1,6)


In [ ]:
map_waveglider_variable('water_temperature',-1,4,wg_number=130)
map_waveglider_variable('air_temperature',-5,5,wg_number=153)

<h2>Make Timeseries Plots on a 2D Axis</h2>
The function in this section plots physical measurement variables collected from the wavegliders on a 4 panneled time series figure.

In [ ]:
def plot_waveglider_timeseries(var1,var2,var3,var4,wg_number='all'): 
    
    [colormap,var1_label] = define_variable_attributes(var1)
    [colormap,var2_label] = define_variable_attributes(var2)
    [colormap,var3_label] = define_variable_attributes(var3)
    [colormap,var4_label] = define_variable_attributes(var4)

    if wg_number == 'all':
        #SECLECT AND LOAD WAVEGLIDER DATA
        file_130 = dir_in+'SASSIE_Fall_2022_Waveglider_130.nc'
        file_153 = dir_in+'SASSIE_Fall_2022_Waveglider_153.nc'
        file_245 = dir_in+'SASSIE_Fall_2022_Waveglider_245.nc'
        file_247 = dir_in+'SASSIE_Fall_2022_Waveglider_247.nc'
        
        ds_130 = xr.open_dataset(file_130)
        ds_153 = xr.open_dataset(file_153)
        ds_245 = xr.open_dataset(file_245)
        ds_247 = xr.open_dataset(file_247)
        
        #MAKE A FOUR PANELED TIME SERIES PLOT
        
        fig,axs = plt.subplots(4,sharex=True,figsize=(10,10))
        
        plt.rcParams['axes.labelsize'] =12
        
        axs[0].plot(ds_130['time'],ds_130[var1][0,:],c='blue',linewidth=0.4,label='WG 130')
        axs[0].plot(ds_153['time'],ds_153[var1][0,:],c='red',linewidth=0.4,label='WG 153')
        axs[0].plot(ds_245['time'],ds_245[var1][0,:],c='green',linewidth=0.4,label='WG 245')
        axs[0].plot(ds_247['time'],ds_247[var1][0,:],c='cyan',linewidth=0.4,label='WG 247')
        axs[0].set(ylabel=var1_label)
        
        axs[0].legend(ncol=2)
        

        axs[1].plot(ds_130['time'],ds_130[var2][0,:],c='blue',linewidth=0.4)
        axs[1].plot(ds_153['time'],ds_153[var2][1,:],c='red',linewidth=0.4)
        axs[1].plot(ds_245['time'],ds_245[var2][0,:],c='green',linewidth=0.4)
        axs[1].plot(ds_247['time'],ds_247[var2][0,:],c='cyan',linewidth=0.4)
        axs[1].set(ylabel=var2_label)
        
        axs[2].plot(ds_130['time'],ds_130[var3],c='blue',linewidth=0.4)
        axs[2].plot(ds_153['time'],ds_153[var3],c='red',linewidth=0.4)
        axs[2].plot(ds_245['time'],ds_245[var3],c='green',linewidth=0.4)
        axs[2].plot(ds_247['time'],ds_247[var3],c='cyan',linewidth=0.4)
        axs[2].set(ylabel=var3_label)
        
        axs[3].plot(ds_130['time'],ds_130[var4],c='blue',linewidth=0.4)
        axs[3].plot(ds_153['time'],ds_153[var4],c='red',linewidth=0.4)
        axs[3].plot(ds_245['time'],ds_245[var4],c='green',linewidth=0.4)
        axs[3].plot(ds_247['time'],ds_247[var4],c='cyan',linewidth=0.4)
        axs[3].set(ylabel=var4_label,xlabel='Date')
        
        plt.xticks(pd.date_range(start=ds_245['time'].values[0],end=ds_245['time'].values[-1],periods=3))
        fig.suptitle(var1_label+', '+var2_label+', '+var3_label+', and '+var4_label+' Data from All Wavegliders',fontsize=13,y=0.94)

    else: 
        #SECLECT AND LOAD WAVEGLIDER DATA
        file = dir_in+'SASSIE_Fall_2022_Waveglider_'+str(wg_number)+'.nc'
        ds = xr.open_dataset(file)

        #MAKE A FOUR PANELED TIME SERIES PLOT
        fig,axs = plt.subplots(4,sharex=True)

        axs[0].plot(ds['time'],ds[var1][0,:],c='black',linewidth=0.4)
        axs[0].set(ylabel=var1_label)
        axs[1].plot(ds['time'],ds[var2][0,:],c='black',linewidth=0.4)
        axs[1].set(ylabel=var2_label)
        axs[2].plot(ds['time'],ds[var3],c='black',linewidth=0.4)
        axs[2].set(ylabel=var3_label)
        axs[3].plot(ds['time'],ds[var4],c='black',linewidth=0.4)
        axs[3].set(ylabel=var4_label,xlabel = 'Date')

        plt.xticks(pd.date_range(start=ds['time'].values[0],end=ds['time'].values[-1],periods=3))
        fig.suptitle(var1_label+', '+var2_label+', and '+var3_label+' Data from Waveglider '+str(wg_number),fontsize=9)

    ##SAVE FIGURE 
    if not os.path.exists(fig_dir+'/timeseries'):
        os.makedirs(fig_dir+'/timeseries')
    print('Saving Output Image:  '+fig_dir+'timeseries/Waveglider'+str(wg_number)+'_'+var1+'_'+var2+'_'+var3+'_'+var4+'.png')
    plt.savefig(fig_dir+'timeseries/Waveglider'+str(wg_number)+'_'+var1+'_'+var2+'_'+var3+'_'+var4+'.png',dpi='figure',format='png')
    

In [ ]:
plot_waveglider_timeseries('water_temperature','salinity','wind_speed','surface_wave_height')